# stablediffusion-infinity

https://github.com/lkwq007/stablediffusion-infinity

Outpainting with Stable Diffusion on an infinite canvas

In [ ]:
#@title setup libs
!nvidia-smi -L
!pip install -qq -U diffusers==0.6.0 transformers ftfy
!pip install -q gradio==3.6
!pip install -q fpie timm

In [ ]:
#@title setup stablediffusion-infinity
!git clone --recurse-submodules https://github.com/lkwq007/stablediffusion-infinity
%cd stablediffusion-infinity
!cp -r PyPatchMatch/csrc .
!cp PyPatchMatch/Makefile .
!cp PyPatchMatch/travis.sh .
!cp PyPatchMatch/patch_match.py . 

In [ ]:
#@title start stablediffusion-infinity (first setup may takes about two minutes for downloading models)
!python app.py --share